In [1]:
import os

In [2]:
%pwd

'c:\\Users\\hp\\Documents\\machine_leanrning\\Chicken-disease-Mlops\\research'

In [3]:
os.chdir("../")

In [4]:
%pwd

'c:\\Users\\hp\\Documents\\machine_leanrning\\Chicken-disease-Mlops'

In [5]:
# First, we update config.yaml
# next, params.yaml
# then we update the config entity

In [6]:
# update the config entity

from dataclasses import dataclass
from pathlib import Path

@dataclass(frozen=True)
class DataIngestionConfig:
    root_dir: Path
    source_URL: str
    local_data_file: str
    unzip_dir: str

In [7]:
# import constants

from cnnClassifier.constants import *
from cnnClassifier.utils.common import read_yaml, create_directories


In [8]:
# update the ConfigurationManager -> configuration.py

class ConfigurationManager:
    def __init__(
        self,
        config_filepath=CONFIG_FILE_PATH,
        params_filepath=PARAMS_FILE_PATH
    ):
        self.config = read_yaml(config_filepath)
        self.param = read_yaml(params_filepath)

        create_directories([Path(self.config.artifacts_root)])

    def get_data_ingestion_config(self) -> DataIngestionConfig:
        config = self.config.data_ingestion

        create_directories([config.root_dir])

        data_ingestion_config = DataIngestionConfig(
            root_dir=Path(config.root_dir),
            source_URL=config.source_URL,
            local_data_file=config.local_data_file,
            unzip_dir=Path(config.unzip_dir)
        )

        return data_ingestion_config

In [9]:
# implement components (Data Ingestion)

import os
import zipfile
import gdown
from cnnClassifier import logger
from cnnClassifier.utils.common import get_size

In [10]:
class DataIngestion:
    def __init__(self, config: DataIngestionConfig):
        self.config = config
    
    def download_data(self) ->  str:
        """
        Fetch data from url
        """

        try:
            dataset_url = self.config.source_URL
            zip_download_dir = self.config.local_data_file
            os.makedirs(self.config.root_dir, exist_ok=True) 
            logger.info(f"Downloading data from {dataset_url} into file {zip_download_dir}")

            file_id = dataset_url.split("/")[-2]
            prefix = "https://drive.google.com/uc?/export=download&id="
            gdown.download(prefix+file_id, zip_download_dir)

            logger.info(f"Downloaded data from {dataset_url} into file {zip_download_dir}")

        except Exception as e:
            raise e
        
    def extract_zip_file(self):
        """
        zip_file_path: str
        Extracts the zip file into data directory
        Function returns None
        """
        unzip_path = self.config.unzip_dir
        os.makedirs(unzip_path, exist_ok=True)
        with zipfile.ZipFile(self.config.local_data_file, 'r') as zip_ref:
            zip_ref.extractall(self.config.unzip_dir)
            

In [12]:

try:
    config = ConfigurationManager()
    data_ingestion_config = config.get_data_ingestion_config()
    data_ingestion = DataIngestion(config=data_ingestion_config)
    data_ingestion.download_data()
    data_ingestion.extract_zip_file()
except Exception as e:
    raise e

[2026-01-16 10:30:03,825: INFO: common: yaml file: config\config.yaml loaded successfully:]
[2026-01-16 10:30:03,829: INFO: common: yaml file: params.yaml loaded successfully:]
[2026-01-16 10:30:03,833: INFO: common: created directory at: artifacts:]
[2026-01-16 10:30:03,835: INFO: common: created directory at: artifacts/data_ingestion:]
[2026-01-16 10:30:03,842: INFO: 1866184008: Downloading data from https://drive.google.com/file/d/1hEIIDJ2ObNRNOlv9J_k2Sdn_9vSUhCs1/view?usp=sharing into file artifacts/data_ingestion/data.zip:]


Downloading...
From: https://drive.google.com/uc?/export=download&id=1hEIIDJ2ObNRNOlv9J_k2Sdn_9vSUhCs1
To: c:\Users\hp\Documents\machine_leanrning\Chicken-disease-Mlops\artifacts\data_ingestion\data.zip
100%|██████████| 5.20M/5.20M [00:02<00:00, 1.84MB/s]


[2026-01-16 10:30:11,661: INFO: 1866184008: Downloaded data from https://drive.google.com/file/d/1hEIIDJ2ObNRNOlv9J_k2Sdn_9vSUhCs1/view?usp=sharing into file artifacts/data_ingestion/data.zip:]
